In [100]:
%config StoreMagics.autorestore = True
%reset -f
# %store -z
%store -r
%whos

Variable           Type              Data/Info
----------------------------------------------
Tsam               float             0.1
dvMathcalP         LinearIOSystem    <LinearIOSystem>: sys[143<...>[[0. 0.]\n     [0. 0.]]\n
extMathcalP        LinearIOSystem    <LinearIOSystem>: sys[142<...> [0. 0.]\n     [0. 0.]]\n
fullOrder          int               84
hsvDvMathcalP      ndarray           84: 84 elems, type `float64`, 672 bytes
hsvExtMathcalP     ndarray           84: 84 elems, type `float64`, 672 bytes
mathcalC           StateSpace        A = [[-1000.]]\n\nB = [[1<...>00.1]]\n\nD = [[100.1]]\n
mathcalP_cb        LinearIOSystem    <LinearIOSystem>: sys[18]<...>[[0. 0.]\n     [0. 0.]]\n
overlineMathcalC   StateSpace        A = [[-0.]]\n\nB = [[10.]<...>= [[100.1]]\n\ndt = 0.1\n
reducOrder         int               10
truncDvMathcalP    LinearIOSystem    <LinearIOSystem>: sys[145<...>[[0. 0.]\n     [0. 0.]]\n
truncExtMathcalP   LinearIOSystem    <LinearIOSystem>: sys[144<...> [0. 0

In [101]:
import numpy
import sympy
import scipy
import control
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

import multiprocessing
from joblib import Parallel, delayed  
from tqdm.notebook import tqdm
import psutil

from custom_utils import ControlUtils
from custom_utils import ODEUtils

from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

numpy.set_printoptions(precision=3)
sympy.init_printing(use_latex='mathjax', use_unicode=True)
sympy.init_printing()
plt.rcParams.update({ "text.usetex": True, "font.family": "Helvetica", "axes.titley": 1.0 } )
%matplotlib inline

In [102]:
s = control.TransferFunction.s
def smallgain_condition(Tsam,mathcalC,extMathcalP,truncExtMathcalP):
        # Norm operator Delta
        normInfDelta = 2*Tsam/numpy.pi
        # Discretizing controller and computing norm of widetildeMathcalCs
        overlineMathcalC = control.tf2ss(control.sample_system(control.ss2tf(mathcalC), Tsam, 'zoh'))
        # overlineMathcalC = control.sample_system(mathcalC, Tsam, 'zoh')
        mathcalCPrime = control.ss(mathcalC.A,mathcalC.B,mathcalC.C,numpy.zeros(mathcalC.D.shape))
        tfMathcalCPrime = control.ss2tf(mathcalCPrime)
        gamma_dv_tildew = normInfDelta * ControlUtils.hinfnorm(mathcalC) \
                                + normInfDelta * ControlUtils.hinfnorm(s * tfMathcalCPrime) * normInfDelta \
                                + ControlUtils.hinfnorm(tfMathcalCPrime) * normInfDelta 
        #######################################################################                        
        # Creating closed-loop
        # Controller matrices
        Ac = mathcalC.A
        Bv = mathcalC.B
        Cw = mathcalC.C
        Dw = mathcalC.D
        #######################################################################
        # FOM extended plant
        Ap = extMathcalP.A
        Bw = extMathcalP.B[:,0:1]
        Bu = extMathcalP.B[:,1:2]
        Cv = extMathcalP.C[0:1,:]
        Cy = extMathcalP.C[-1:,:]
        Dv = extMathcalP.D[0:1,:]
        Dy = extMathcalP.D[-1:,:]
        # Closed-loop with FOM Extended Plant
        extAcl = numpy.row_stack( [ numpy.column_stack( [ Ap + Bw@Dw@Cv, Bw@Cw ] ),
                                    numpy.column_stack( [ Bv@Cv, Ac ] )
                                    ] )
        extBcl = numpy.row_stack( [ numpy.column_stack([Bw, Bu]),
                                    numpy.column_stack( [ numpy.zeros( (Ac.shape[0],Bw.shape[1]) ), numpy.zeros( (Ac.shape[0],Bu.shape[1]) ) ] )
                                    ] )
        extCcl = numpy.row_stack( [ numpy.column_stack( [ Cv, numpy.zeros( (Cv.shape[0],Ac.shape[0]) ) ] ),
                                    numpy.column_stack( [ Cv@Ap + Cv@Bw@Dw@Cv, Cv@Bw@Cw ] ),
                                    numpy.column_stack( [ Cy, numpy.zeros( (Cy.shape[0],Ac.shape[0]) ) ] )
                                    ] )
        extDcl = numpy.row_stack( [ numpy.column_stack( [ numpy.zeros( (Cv.shape[0],Bw.shape[1]) ), numpy.zeros( (Cv.shape[0],Bu.shape[1]) ) ] ),
                                    numpy.column_stack( [ Cv@Bw , Cv@Bu] ),
                                    numpy.column_stack( [ numpy.zeros( (Cy.shape[0],Bw.shape[1]) ), numpy.zeros( (Cy.shape[0],Bu.shape[1]) ) ] )
                                    ] )
        extMathcalPcl = control.ss(extAcl,extBcl,extCcl,extDcl)
        #######################################################################
        # ROM extended plant
        truncExtAp = truncExtMathcalP.A
        truncExtBw = truncExtMathcalP.B[:,0:1]
        truncExtBu = truncExtMathcalP.B[:,1:2]
        truncExtCv = truncExtMathcalP.C[0:1,:]
        truncExtCy = truncExtMathcalP.C[-1:,:]
        truncExtDv = truncExtMathcalP.D[0:1,:]
        truncExtDy = truncExtMathcalP.D[-1:,:]
        # Closed-loop with ROM Extended Plant
        truncExtAcl = numpy.row_stack( [ numpy.column_stack( [ truncExtAp + truncExtBw@Dw@truncExtCv, truncExtBw@Cw ] ),
                                    numpy.column_stack( [ Bv@truncExtCv, Ac ] )
                                    ] )
        truncExtBcl = numpy.row_stack( [ numpy.column_stack([truncExtBw, truncExtBu]),
                                    numpy.column_stack( [ numpy.zeros( (Ac.shape[0],truncExtBw.shape[1]) ), numpy.zeros( (Ac.shape[0],truncExtBu.shape[1]) ) ] )
                                    ] )
        truncExtCcl = numpy.row_stack( [ numpy.column_stack( [ truncExtCv, numpy.zeros( (truncExtCv.shape[0],Ac.shape[0]) ) ] ),
                                    numpy.column_stack( [ truncExtCv@truncExtAp + truncExtCv@truncExtBw@Dw@truncExtCv, truncExtCv@truncExtBw@Cw ] ),
                                    numpy.column_stack( [ truncExtCy, numpy.zeros( (truncExtCy.shape[0],Ac.shape[0]) ) ] )
                                    ] )
        truncExtDcl = numpy.row_stack( [ numpy.column_stack( [ numpy.zeros( (truncExtCv.shape[0],truncExtBw.shape[1]) ), numpy.zeros( (truncExtCv.shape[0],truncExtBu.shape[1]) ) ] ),
                                    numpy.column_stack( [ truncExtCv@truncExtBw , truncExtCv@truncExtBu] ),
                                    numpy.column_stack( [ numpy.zeros( (truncExtCy.shape[0],truncExtBw.shape[1]) ), numpy.zeros( (truncExtCy.shape[0],truncExtBu.shape[1]) ) ] )
                                    ] )
        truncExtMathcalPcl = control.ss(truncExtAcl,truncExtBcl,truncExtCcl,truncExtDcl)
        #######################################################################
        approxErrorExtMathcalPcl = extMathcalPcl-truncExtMathcalPcl
        #######################################################################
        # Computing norms for small-gain stability condition
        gamma_tildew_dv = ControlUtils.hinfnorm(extMathcalPcl[1,0])
        epsilon_tildew_dv = ControlUtils.hinfnorm(approxErrorExtMathcalPcl[1,0])
        return (gamma_tildew_dv,epsilon_tildew_dv,gamma_dv_tildew,extMathcalPcl,truncExtMathcalPcl,approxErrorExtMathcalPcl)

In [103]:
# Loop parameters
kp_kd_steps = 10
kps = numpy.linspace(50e-6, 100.0e-6, kp_kd_steps)
tds = numpy.linspace(1.0e5, 1.0e5, kp_kd_steps)
N = 1.0e3

samplings_lims = [50.0e-3,100.0e-3]
samplings_steps = 10
samplings = numpy.linspace(samplings_lims[0],samplings_lims[1],samplings_steps)

idx_args_list = []
for i in range(samplings_steps):
    for j in range(kp_kd_steps):
        Tsam = samplings[i]
        Kp = kps[j]
        Td = tds[j]

        # I need to rewrite num and den expression as function of kp kd ratio
        num =  [ (Kp/Td) * N * (N + Td), (Kp/Td) * N * (1.0/Td) ]
        den = [ 1, N/Td ]
        mathcalC = control.tf2ss(num,den)

        idx_args_list.append([(i,j),(Tsam,mathcalC,extMathcalP,truncExtMathcalP)])
        
num_cores = multiprocessing.cpu_count()
idx_results_list = Parallel(n_jobs=num_cores)\
                        (\
                            delayed( lambda idx,args: (idx, smallgain_condition(*args)) )\
                                ( idx_args_list[k][0], idx_args_list[k][1] )\
                                for k in tqdm(range(len(idx_args_list)), position=0)\
                        )
                

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
for k in range(len(idx_results_list)):
    (i,j),(gamma_tildew_dv,epsilon_tildew_dv,gamma_dv_tildew,extMathcalPcl,truncExtMathcalPcl,approxErrorExtMathcalPcl) = idx_results_list[k]
    small_gain_cond_FOM = gamma_tildew_dv*gamma_dv_tildew
    small_gain_cond_ROM = (gamma_tildew_dv+epsilon_tildew_dv)*gamma_dv_tildew
    
    Tsam = samplings[i]
    Kp = kps[j]
    Td = tds[j]
    kp_td = Kp/Td
    
    if(small_gain_cond_FOM>=1):
        plt.plot(kp_td, Tsam, marker="x", markersize=4, markerfacecolor='r', markeredgecolor='r')
    elif(small_gain_cond_ROM>=1):
        plt.plot(kp_td, Tsam, marker="x", markersize=4, markerfacecolor='b', markeredgecolor='b')
    else:
        plt.plot(kp_td, Tsam, marker="o", markersize=4, markerfacecolor='none', markeredgecolor='g')

plt.ticklabel_format(axis='y', scilimits=(-3,-3))
plt.ticklabel_format(axis='x', scilimits=(-6,-6))
plt.ylabel('$T$')
plt.xlabel('$\\frac{K_p}{T_d}$')
plt.show()

In [105]:
%store -d idx_results_list
%store idx_results_list

Stored 'idx_results_list' (list)
